In [ ]:
#@title Dependencies
 # Equilibrium HQS
!pip install ttictoc
from ttictoc import tic,toc
import torch

import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.autograd as autograd
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from scipy.io import loadmat
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import image
from pathlib import Path
from PIL import Image
import random, math
import numpy as np
!pip install imagesc
import imagesc as imagesc
import scipy
from torch.autograd import Variable
from torch.nn import Linear, ReLU, LeakyReLU, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import glob
!pip install mat73
import mat73
import random
import sys
import os
!pip install -U scikit-image
import skimage
from functools import partial
import cv2
from sklearn.linear_model import OrthogonalMatchingPursuit
from math import log10, sqrt
from google.colab import drive
from sklearn import preprocessing
from skimage.metrics import structural_similarity as ssim
drive.mount("/content/drive/", force_remount=True)
py_file_location = "/content/drive/My Drive/Colab Notebooks/DEEP_UNROLLING_phd/MY_PROPOSED_MODELS_PAPER"
sys.path.append(os.path.abspath(py_file_location))


# Set the GPu as default
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#------------------------------------------------------------------------------#




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 KB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=a6f983a25b587ba1606f060c5bd6ed0e7c2a835163c2431379f588478be76194
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:

KeyboardInterrupt: ignored

In [ ]:
#@title Read Data and Testing Functions
def read_test_hyperspectral_images(image_path, indx):
    os.chdir(image_path)
    mat_files = glob.glob('*.mat')
    fname = mat_files[indx]
    print(fname)
    matData = mat73.loadmat(fname)
    spectra = matData['rad']
    spectra = spectra[0:1000, 0:1000,:]
    #spectra = min_max_normalized(spectra)
    spectra = (spectra-np.amin(spectra))/(np.amax(spectra)-np.amin(spectra))
    return spectra

def myPSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = np.amax(original)
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def add_gaussian_noise_matrix_form(Hd, sigma):
    [N,M,B] = Hd.shape
    noise = sigma/255*np.random.randn(N, M, B);
    Hd_noisy = Hd + noise
    return Hd_noisy


def add_gaussian_noise_matrix_form_colored_stripe(Hd, sigma):
  [N,M,B] = Hd.shape
  Hd_noisy = np.zeros(Hd.shape)
  for i in range(Hd.shape[2]):

        sigma = [5, 10, 10, 30, 40, 60]
        noise = random.choice(sigma)/255*np.random.randn(N, M);
        Hd_noisy[:,:,i] = Hd[:,:,i] + noise

  min_amount = 0.05;
  max_amount = 0.15;
  band = np.random.permutation(B);
  band = band[0:10];
  stripnum = np.random.randint(math.ceil(min_amount * N), math.ceil(max_amount * N), band.shape[0]);
  for i in range(band.shape[0]):
    loc = np.random.permutation(N);
    a = stripnum[i]
    loc = loc[1:a];
    stripe = np.random.rand(1, loc.shape[0])*0.5-0.25;
    Hd_noisy[:,loc,band[i]] = Hd_noisy[:,loc,band[i]] -stripe


  return Hd_noisy

def add_gaussian_noise_matrix_form_colored_deadline(Hd, sigma):
  [N,M,B] = Hd.shape
  Hd_noisy = np.zeros(Hd.shape)
  for i in range(Hd.shape[2]):

        sigma = [5, 10, 10, 30, 40, 60]
        noise = random.choice(sigma)/255*np.random.randn(N, M);
        Hd_noisy[:,:,i] = Hd[:,:,i] + noise

  min_amount = 0.05;
  max_amount = 0.15;
  band = np.random.permutation(B);
  band = band[0:10];
  stripnum = np.random.randint(math.ceil(min_amount * N), math.ceil(max_amount * N), band.shape[0]);
  for i in range(band.shape[0]):
    loc = np.random.permutation(N);
    a = stripnum[i]
    loc = loc[1:a];
    stripe = np.random.rand(1, loc.shape[0])*0.5-0.25;
    # Hd_noisy[:,loc,band[i]] = Hd_noisy[:,loc,band[i]] -stripe
    Hd_noisy[:,loc,band[i]] = 0

  return Hd_noisy

def construct_the_training_dataset(num_of_patches, patch_size, sparsity_level, D, image_path, noIm_start, noIm_end, sigma, gaus_sigma, rcond):
    # In this function random patches are extracted from a hyperspectral image
    # and its noisy version and the sparse coding matrices are estimated
    Ds_sup = []
    Noisy_data = []
    Clean_data = []
    cnt = 1
    for indx in range(noIm_start, noIm_end):
       print(indx)
       # Read a hyperspectral Image
       X = read_test_hyperspectral_images(image_path, indx)
       X_noisy = add_gaussian_noise_matrix_form(X, sigma)
      #  X_noisy = add_gaussian_noise_matrix_form_colored_stripe(X, sigma)
      #  X_noisy = add_gaussian_noise_matrix_form_colored_deadline(X, sigma)
       cnt, support_patch, data_patch, clean_patch = extract_from_random_patches_SCM(X, X_noisy,
                                                  num_of_patches, patch_size, sparsity_level, D, cnt, rcond)
       if indx == noIm_start:
         Ds_sup = support_patch
         Noisy_data = data_patch
         Clean_data = clean_patch
       else:
         Ds_sup = np.concatenate((Ds_sup, support_patch))
         Noisy_data = np.concatenate((Noisy_data, data_patch))
         Clean_data = np.concatenate((Clean_data, clean_patch))
       del X, X_noisy
    return  Ds_sup, Noisy_data, Clean_data

def extract_from_random_patches_SCM(X, X_noisy, num_of_patches, patch_size, sparsity_level, D, cnt, rcondition):
    # In this function random patches are extracted from a hyperspectral image
    # and its noisy version and the sparse coding matrices are estimate
    image_size = X.shape
    rI = random.sample(range(0, image_size[1]//patch_size-1), num_of_patches)
    rJ = random.sample(range(0, image_size[1]//patch_size-1), num_of_patches)
    #G2d_list = []
    #G2d_noisy_list = []
    dictionary_list = []
    data_list = []
    data_clean = []

    for xx in range(num_of_patches):
        for yy in range(num_of_patches):
            i = rI[xx]
            j = rJ[yy]
            x = X[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size, :]
            x = (np.reshape(x, (patch_size*patch_size, image_size[2]))).T
            x_noisy = X_noisy[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size, :]
            x_noisy = (np.reshape(x_noisy, (patch_size*patch_size, image_size[2]))).T
            #Normlaize the data
            #norm_x_noisy = (np.sum(x_noisy**2, axis=0))**0.5;
            #x_noisy = np.divide(x_noisy, norm_x_noisy)
            #x = np.divide(x, norm_x_noisy)
            #Find the support set based on OMP algorithm and the centroid signal
            x_noisy_mean = np.mean(x_noisy, axis=1)
            omp = OrthogonalMatchingPursuit(n_nonzero_coefs=sparsity_level, fit_intercept=False, normalize=False)
            omp.fit(D, x_noisy_mean)
            coef = omp.coef_
            sup, = coef.nonzero()
            #g = np.dot(np.linalg.pinv(D[:,sup], rcond=rcondition, hermitian=False),x)
            #g_noisy = np.dot(np.linalg.pinv(D[:,sup], rcond=rcondition, hermitian=False),x_noisy)
            #g_noisy= np.ones((sparsity_level,patch_size*patch_size))
            #g = np.reshape(g, (sparsity_level, patch_size, patch_size))
            #g_noisy = np.reshape(g_noisy, (sparsity_level, patch_size, patch_size))
            #G2d_list.append(g)
            #G2d_noisy_list.append(g_noisy)
            dictionary_list.append(D[:,sup])
            data_list.append(x_noisy)
            data_clean.append(x)
            cnt = cnt + 1
    return   cnt, dictionary_list, data_list, data_clean

def reconstruct_hyperspectral_image(X, X_noisy, patch_size, sparsity_level, D, my_sc_model, rcodition, device, batch_size):
    # In this function random patches are extracted from an hyperspectral image
    # and its noisy version and the sparse coding matrices are estimated
    image_size = X.shape
    num_of_patch_i = image_size[0]//patch_size
    num_of_patch_j = image_size[1]//patch_size
    hd=np.zeros((image_size[0], image_size[1], image_size[2]))
    Data = []
    Dictionary = []
    Norm = []
    for i in range(num_of_patch_i):
        for j in range(num_of_patch_j):
            x_noisy = X_noisy[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size, :]
            x_noisy = (np.reshape(x_noisy, (patch_size*patch_size, image_size[2]))).T
            #Normlaize the data
            #norm_x_noisy = (np.sum(x_noisy**2, axis=0))**0.5;
            #Find the support set based on OMP algorithm and the centroid signal
            x_noisy_mean = np.mean(x_noisy, axis=1)
            omp = OrthogonalMatchingPursuit(n_nonzero_coefs=sparsity_level, fit_intercept=False, normalize=False)
            omp.fit(D, x_noisy_mean)
            coef = omp.coef_
            sup, = coef.nonzero()
            # Forward model
            Ds = D[:,sup]
            data = x_noisy
            Dictionary.append(Ds)
            Data.append(data)
            #Norm.append(norm_x_noisy)
    del data
    Data = np.array(Data)
    Dictionary = np.array(Dictionary)
    #Norm = np.array(Norm)

    dataloader_Dictionary = DataLoader(torch.from_numpy(Dictionary), batch_size, shuffle=False)
    dataloader_Data = DataLoader(torch.from_numpy(Data), batch_size, shuffle=False)
    #dataloader_Norm = DataLoader(torch.tensor(Norm), batch_size, shuffle=False)
    del Data, Dictionary
    Rec_Im = []
    cc = 0
    my_sc_model.eval()
    for data, Ds in zip(dataloader_Data, dataloader_Dictionary):
        batch_size = len(data)
        #norm = norm.numpy()
        with torch.no_grad():
          data = torch.autograd.Variable((data.float()).to(device))
          Ds = torch.autograd.Variable((Ds.float()).to(device))


        G = my_sc_model(data , Ds, len(data) , device)
        #norm = np.reshape(norm, (batch_size, 1, patch_size*patch_size))
        im = G.cpu().detach().numpy()
        del G
        im = np.reshape(im, (batch_size, data.shape[1], patch_size*patch_size))
        if cc == 0:
           Rec_Im = im
        else:
           Rec_Im = np.append(Rec_Im, im, axis=0)
        cc = cc + 1
        del data, Ds
    cnt = 0
    for i in range(num_of_patch_i):
       for j in range(num_of_patch_j):
          hd[i*patch_size:(i+1)*patch_size,j*patch_size:(j+1)*patch_size,:]= np.reshape((Rec_Im[cnt,:,:]).T, (patch_size, patch_size, 31))
          cnt = cnt + 1
    del Rec_Im, dataloader_Dictionary, dataloader_Data
    return hd


#Metrics

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

class Bandwise(object):
    def __init__(self, index_fn):
        self.index_fn = index_fn

    def __call__(self, X, Y):
        C = X.shape[2]
        bwindex = []
        for ch in range(C):
            x = torch.squeeze(X[:,:,ch]).cpu().numpy()
            y = torch.squeeze(Y[:,:,ch]).cpu().numpy()
            index = self.index_fn(x, y)
            bwindex.append(index)
        return bwindex

cal_bwssim = Bandwise(partial(ssim))
cal_bwpsnr = Bandwise(partial(skimage.metrics.peak_signal_noise_ratio, data_range=255))


def cal_sam(X, Y, eps=1e-8):
    X = X.cpu().numpy()
    Y = Y.cpu().numpy()
    tmp = (np.sum(X*Y, axis=2) + eps) / (np.sqrt(np.sum(X**2, axis=2)) + eps) / (np.sqrt(np.sum(Y**2, axis=2)) + eps)
    return np.mean(np.real(np.arccos(tmp)))


def MSIQA(X, Y):
    psnr = np.mean(cal_bwpsnr(X, Y))
    ssim = np.mean(cal_bwssim(X, Y))
    sam = cal_sam(X, Y)
    return psnr, ssim, sam

In [ ]:
#@title Deep Equilibrium Model
def anderson(f, x0, m=5, lam=1e-4, max_iter=50, tol=1e-2, beta = 1.0):
    """ Anderson acceleration for fixed point iteration. """

    #x0:Z_initial denotes the initial point to start
    bsz, d, H, W = x0.shape
    X = torch.zeros(bsz, m, d*H*W, dtype=x0.dtype, device=x0.device)
    F = torch.zeros(bsz, m, d*H*W, dtype=x0.dtype, device=x0.device)
    X[:,0], F[:,0] = x0.view(bsz, -1), f(x0).view(bsz, -1)
    X[:,1], F[:,1] = F[:,0], f(F[:,0].view_as(x0)).view(bsz, -1)

    H = torch.zeros(bsz, m+1, m+1, dtype=x0.dtype, device=x0.device)
    H[:,0,1:] = H[:,1:,0] = 1
    y = torch.zeros(bsz, m+1, 1, dtype=x0.dtype, device=x0.device)
    y[:,0] = 1

    res = []
    for k in range(2, 10):
        n = min(k, m)
        G = F[:,:n]-X[:,:n]
        H[:,1:n+1,1:n+1] = torch.bmm(G,G.transpose(1,2)) + lam*torch.eye(n, dtype=x0.dtype, device=x0.device)[None]
        alpha = torch.linalg.solve(H[:,:n+1,:n+1], y[:,:n+1])[:, 1:n+1, 0]   # (bsz x n)


        X[:,k%m] = beta * (alpha[:,None] @ F[:,:n])[:,0] + (1-beta)*(alpha[:,None] @ X[:,:n])[:,0]
        F[:,k%m] = f(X[:,k%m].view_as(x0)).view(bsz, -1)
        res.append((F[:,k%m] - X[:,k%m]).norm().item()/(1e-5 + F[:,k%m].norm().item()))
        if (res[-1] < tol):
            break
    #print(res[-1])
    return X[:,k%m].view_as(x0), res


class HQS_Module(nn.Module):
    def __init__(self, sparsity, patch_size, init_value, DnCnn):
        super().__init__()

        self.DnCnn = DnCnn
        self.alpha = nn.Parameter(torch.tensor(init_value), requires_grad=True)
        self.sparsity = sparsity
        self.patch_size = patch_size

    # Defining the forward pass
    def forward(self, z, data, dictionary, batch_size, device):
         z = z.to(device)
         z=torch.reshape(z,(batch_size, dictionary.shape[1], patch_size*patch_size))

         ii=torch.eye(dictionary.shape[2])
         I=(ii.unsqueeze(0).repeat(batch_size, 1, 1)).to(device)

         DT = torch.transpose(dictionary,1,2)
         G = torch.matmul(DT, dictionary)
         DtD = torch.inverse(G + G)
         A = torch.matmul(DT, data)
         A = self.alpha*A
         x = torch.matmul(DtD, (A + torch.matmul(DT, z)))
         z = (torch.matmul(dictionary, x)).to(device)
         z = torch.reshape(z, (batch_size, dictionary.shape[1], patch_size, patch_size))
         z = self.DnCnn(z)

         #Output
         #z=torch.reshape(z, (batch_size, sparsity_level, patch_size*patch_size))
         return z

class DEQFixedPoint(nn.Module):
    def __init__(self, f, sparsity, patch_size, device, solver, **kwargs):
        super().__init__()
        self.f = f
        self.solver = solver
        self.kwargs = kwargs
        self.sparsity = sparsity
        self.patch_size = patch_size
    def forward(self, data, dictionary, batch_size, device):
        # compute forward pass and re-engage autograd tape
        with torch.no_grad():
            z, self.forward_res = self.solver(lambda z : self.f(z, data, dictionary, batch_size, device), torch.ones(batch_size, 31, patch_size, patch_size), **self.kwargs)
        z = self.f(z, data, dictionary, batch_size, device)

        # set up Jacobian vector product (without additional forward calls)
        z0 = z.clone().detach().requires_grad_()
        f0 = self.f(z0, data, dictionary, batch_size, device)
        def backward_hook(grad):
            g, self.backward_res = self.solver(lambda y : autograd.grad(f0, z0, y, retain_graph=True)[0] + grad,
                                               grad, **self.kwargs)
            return g
        z.register_hook(backward_hook)
        return z

def equilibrium_model_training(my_sc_model, num_epochs, learning_rate, dataloader_train,
                               D, device, l2_w, sparsity_level, batch_size,
                               patch_size):
    criterion = nn.MSELoss()
    #criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(my_sc_model.parameters(), lr=learning_rate, weight_decay=l2_w)
    for epoch in range(num_epochs):
       acc_loss = 0.
       for Ds, data, clean in dataloader_train:

           batch_size = len(data)
           data = (data.float()).to(device)
           Ds = (Ds.float()).to(device)
           clean = (clean.float()).to(device)
           clean = torch.reshape(clean, (batch_size, Ds.shape[1], patch_size, patch_size))
           # ===================forward=====================
           output = my_sc_model(data, Ds, batch_size, device)
           loss = torch.sqrt(criterion(output, clean))
           #loss = criterion(output, clean)
           # ===================backward====================
           optimizer.zero_grad()
           loss.backward()
           optimizer.step()
           acc_loss += loss.item()
       #if epoch%10 == 0:
         #torch.save(my_sc_model,"/content/drive/MyDrive/Colab Notebooks/DEEP_UNROLLING_phd/models/fast_eq_50_50x50.pt")

       print(acc_loss / len(dataloader_train))


In [ ]:
#@title Pretraining DnCNN
class DnCnn_net(nn.Module):
       def __init__(self, num_of_layers, features, input_channels):
         super().__init__()
         kernel_size = 3
         padding = 1
         features = features
         channels = input_channels
         num_of_layers = num_of_layers
         layers = []
         layers.append(nn.utils.spectral_norm(nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False)))
         layers.append(nn.ReLU(inplace=True))
         for _ in range(num_of_layers-2):
             layers.append(nn.utils.spectral_norm(nn.Conv2d(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False)))
             #layers.append(nn.BatchNorm2d(features))
             layers.append(nn.ReLU())
         layers.append(nn.utils.spectral_norm(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False)))
         self.dncnn = (nn.Sequential(*layers))

       def forward(self, input):
           return self.dncnn(input)

def pretraining(DnCnn_model, num_epochs, learning_rate, dataloader_pretrain, D, device, l2_w,
                 sparsity_level, batch_size, patch_size):
     criterion = nn.MSELoss()
     #criterion = nn.L1Loss()
     optimizer = torch.optim.Adam(DnCnn_model.parameters(), lr=learning_rate, weight_decay=l2_w)

     for epoch in range(num_epochs):
        acc_loss = 0.
        for s, data_noisy, clean in dataloader_pretrain:
            # if batch_size>1:
            batch_size = len(data_noisy)
            data_noisy = (data_noisy.float()).to(device)
            data_noisy = torch.reshape(data_noisy, (batch_size, clean.shape[1], patch_size, patch_size))
            clean = (clean.float()).to(device)
            clean = torch.reshape(clean, (batch_size, clean.shape[1], patch_size, patch_size))
            # ===================forward=====================
            output = DnCnn_model(data_noisy)
            loss = torch.sqrt(criterion(output, clean))
            #loss = (criterion(output, clean))
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc_loss += loss.item()
        print(acc_loss / len(dataloader_pretrain))

In [ ]:
#@title Training Phase
#Read the dictionary
matData = scipy.io.loadmat("/content/drive/MyDrive/DATA_ICVL/dictionary_test_1.mat")
D = matData['D']

# X = scipy.io.loadmat("/content/drive/MyDrive/DATA_ICVL/Indian_pines.mat")
# X = X['indian_pines']
#Parameters
image_path = "/content/drive/MyDrive/DATA_ICVL/Train_colored"
num_of_patches = 9
patch_size = 50
sparsity_level = 12
noIm_start = 0
noIm_end = 50
sigma = 2
gaus_sigma = 0.000005
rcond = 1e-05

# create the training dataset
train_Support, train_Data, train_clean = construct_the_training_dataset(
    num_of_patches, patch_size, sparsity_level, D, image_path, noIm_start, noIm_end, sigma, gaus_sigma, rcond)

train_Support = np.float32(train_Support)
train_Data = np.float32(train_Data)
train_clean = np.float32(train_clean)

# Create the dataloader
batch_size = 6
train_ds = TensorDataset(torch.from_numpy(train_Support), torch.from_numpy(train_Data), torch.from_numpy(train_clean))
dataloader_train = DataLoader(train_ds, batch_size, shuffle=False)
del train_ds
del train_Support, train_Data, train_clean

#Pretrain the DnCnn model
l2_w = 1e-08
num_epochs = 50
learning_rate = 1e-3

#Denoiser and pretraining
DnCnn_model = DnCnn_net(5, 64, 31).to(device)
# DnCnn_model = MemNet(31, 64, 5, 3).to(device )
# pretraining(DnCnn_model, num_epochs, learning_rate, dataloader_train, D, device, l2_w, sparsity_level,
#                                batch_size, patch_size)
# learning_rate = 1e-4
# pretraining(DnCnn_model, num_epochs, learning_rate, dataloader_train, D, device, l2_w, sparsity_level,
#                                batch_size, patch_size)

#DeQ Model
l2_w = 1e-08
num_epochs = 50
learning_rate = 1e-4
patch_size = 50
init_value = 10.99
eq_module = HQS_Module(sparsity_level, patch_size, init_value, DnCnn_model).to(device)
my_sc_model = DEQFixedPoint(eq_module, sparsity_level, patch_size, device, anderson, m=5, lam=1e-03, max_iter=25, tol=1e-3, beta=1.0).to(device)

learning_rate = 1e-4
num_epochs = 80
equilibrium_model_training(my_sc_model, num_epochs, learning_rate, dataloader_train,
                           D, device, l2_w, sparsity_level, batch_size,
                           patch_size)


In [ ]:
#@title Testing Phase
#--------------------------------------------------------------------------------------------------------------#

sigma = 50
patch_size = 100
#Testing
image_test_path = "/content/drive/MyDrive/DATA_ICVL/Test"
psnr_n1 = []
sim_n = []
sam_n = []
time_rec = []

# my_sc_model=torch.load("/content/drive/MyDrive/Colab Notebooks/DEEP_UNROLLING_phd/models/fast_eq_30_50x50.pt")

for kk in range (42):
    print(kk)
    #Test reconstruction
    X = read_test_hyperspectral_images(image_test_path, kk)
    X_noisy = add_gaussian_noise_matrix_form(X, sigma)

    # X_noisy = add_gaussian_noise_matrix_form_colored(X, sigma)
    # X_noisy = add_gaussian_noise_matrix_form_colored_stripe(X, sigma)

    # X_noisy = add_gaussian_noise_matrix_form_colored_deadline(X, sigma)


    # X_noisy = scipy.ndimage.gaussian_filter(X_noisy, 0.005)

    tic()
    X_recon = reconstruct_hyperspectral_image(X, X_noisy, patch_size, sparsity_level,
                                              D, my_sc_model, rcond, device, 5)
    time_rec.append((toc()))

    X_recon[X_recon>1]=1
    X_recon[X_recon<0]=0

    a, b, c = MSIQA(torch.from_numpy(X*255), torch.from_numpy(X_recon*255))

    psnr_n1.append(a)
    sim_n.append(b)
    sam_n.append(c)

print(np.mean(psnr_n1))
print(np.mean(sim_n))
print(np.mean(sam_n))
print(np.mean(time_rec))

# kk=10
# fig = plt.figure(figsize=(15,10))
# plt.imshow(np.uint8(X_recon[:,:,23]*255), cmap='gray', aspect='auto')
# fig = plt.figure(figsize=(15,10))
# plt.imshow(np.uint8(X_noisy[:,:,23]*255), cmap='gray', aspect='auto')
# fig = plt.figure(figsize=(15,10))
# plt.imshow(np.uint8(X[:,:,23]*255), cmap='gray', aspect='auto')

# [50:1050-50, 50:1050-50, :]
# [50:1050-50, 50:1050-50, :]
a, b, c = MSIQA(torch.from_numpy(X*255), torch.from_numpy(X_noisy*255))


0
bgu_0403-1525.mat
1
eve_0331-1549.mat
2
bulb_0822-0909.mat
3
BGU_0522-1211.mat
4
BGU_0522-1201.mat
5
bgu_0403-1511.mat
6
bgu_0403-1444.mat
7
BGU_0403-1419-1.mat
8
bguCAMP_0514-1724.mat
9
bguCAMP_0514-1723.mat
10
bguCAMP_0514-1718.mat
11
bguCAMP_0514-1659.mat
12
eve_0331-1606.mat
13
eve_0331-1633.mat
14
eve_0331-1646.mat
15
eve_0331-1647.mat
16
gavyam_0823-0930.mat
17
hill_0325-1235.mat
18
hill_0325-1228.mat
19
gavyam_0823-0944.mat
20
gavyam_0823-0933.mat
21
Lehavim_0910-1630.mat
22
Lehavim_0910-1626.mat
23
lehavim_0910-1610.mat
24
lehavim_0910-1607.mat
25
nachal_0823-1145.mat
26
nachal_0823-1144.mat
27
nachal_0823-1047.mat
28
nachal_0823-1038.mat
29
nachal_0823-1147.mat
30
omer_0331-1131.mat
31
omer_0331-1104.mat
32
objects_0924-1602.mat
33
objects_0924-1557.mat
34
objects_0924-1556.mat
35
objects_0924-1550.mat
36
nachal_0823-1222.mat
37
peppers_0503-1315.mat
38
rsh_0406-1413.mat
39
prk_0328-1031.mat
40
plt_0411-1155.mat
41
pepper_0503-1236.mat
38.99499162073993
0.9505339320805037
0.